[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [0]:
!git clone https://github.com/eth-sri/ERAN.git

In [0]:
%cd ERAN

### Install M4

In [0]:
!wget ftp://ftp.gnu.org/gnu/m4/m4-1.4.1.tar.gz
!tar -xvzf m4-1.4.1.tar.gz

In [0]:
%cd m4-1.4.1

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
!cp src/m4 /usr/bin

In [0]:
%cd ..

In [0]:
!rm m4-1.4.1.tar.gz

### Install gmp

In [0]:
!wget https://gmplib.org/download/gmp/gmp-6.1.2.tar.xz
!tar -xvf gmp-6.1.2.tar.xz

In [0]:
%cd gmp-6.1.2

In [0]:
!./configure --enable-cxx

In [0]:
!make
!make install

In [0]:
!make check

In [0]:
%cd ..

In [0]:
!rm gmp-6.1.2.tar.xz

### Install mpfr

In [0]:
!wget https://www.mpfr.org/mpfr-current/mpfr-4.0.2.tar.xz
!tar -xvf mpfr-4.0.2.tar.xz

In [0]:
%cd mpfr-4.0.2

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
%cd ..

In [0]:
!rm mpfr-4.0.2.tar.xz

### Install ELINA

In [0]:
!git clone https://github.com/eth-sri/ELINA.git

In [0]:
%cd ELINA

In [0]:
!./configure

In [0]:
!make
!make install

In [0]:
%cd ..

### Install Gurobi

In [0]:
!wget https://packages.gurobi.com/9.0/gurobi9.0.0_linux64.tar.gz
!tar -xvf gurobi9.0.0_linux64.tar.gz

In [0]:
%cd gurobi900/linux64/src/build

In [0]:
!sed -ie 's/^C++FLAGS =.*$/& -fPIC/' Makefile
!make

In [0]:
!cp libgurobi_c++.a ../../lib/

In [0]:
%cd ../../

In [0]:
!cp lib/libgurobi90.so /usr/lib
!python3 setup.py install

In [0]:
%cd ../../

### Update environment variables:

In [0]:
%env GUROBI_HOME=/content/ERAN/gurobi900/linux64
gurobi_home = %env GUROBI_HOME
%env PATH
path = %env PATH
path += ':' + gurobi_home
print(path)
%env PATH=$path
ld_library_path = %env LD_LIBRARY_PATH
ld_library_path += ':' + gurobi_home
print(ld_library_path)
%env LD_LIBRARY_PATH=$ld_library_path

### Install DeepG

In [0]:
!rm -r deepg
!git clone https://github.com/eth-sri/deepg.git

In [0]:
%cd deepg/code

In [0]:
!sed s/81/90/ <Makefile >Makefile2
!sed s/build/./ <Makefile2 >Makefile3
!rm Makefile Makefile2
!mv Makefile3 Makefile

In [0]:
!mkdir build
!make shared_object

In [0]:
!cp ./build/libgeometric.so /usr/lib
!cd ../..

In [0]:
!ldconfig


### Install other requirements


In [0]:
%cd ../..

In [0]:
!sed "s/tensorflow/tensorflow<2/" < requirements.txt >requirements2.txt
!rm requirements.txt
!mv requirements2.txt requirements.txt
!pip3 install -r requirements.txt

---
----

In [0]:
#%cd /content

In [0]:
#!mkdir /content/test_output

In [0]:
#!wget -O mnist.tf https://files.sri.inf.ethz.ch/eran/nets/tensorflow/mnist/mnist_relu_3_50.tf

## Differential 

In [0]:
%cd /content

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# try:
#     %tensorflow version 1.x
# except Exception:
#     pass

%tensorflow_version 1.x

import tensorflow as tf
import numpy as np
import os

if not os.path.exists("/content/model_ckpts"):
    os.makedirs("/content/model_ckpts")

In [0]:
!pip install tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer
#from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPAdamGaussianOptimizer

## Loading Data

In [0]:
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype = np.float32)/255
test_data = np.array(test_data, dtype=np.float32)/255

train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

train_labels = np.array(train_labels, dtype= np.int32)
test_labels = np.array(test_labels, dtype= np.int32)


### Params

In [0]:
epochs = 15
batch_size = 250

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multuple of the number of microbatches')

### Data Prep

In [0]:
def create_tf_file(model, path):
    # Create the .tf file manually as expected by ERAN
    with open(path, 'w') as f:
        for layer in model.layers:
            if isinstance(layer, tf.keras.layers.Conv2D):
                f.write('Conv2D\n')
                activation = tf.keras.activations.serialize(layer.activation)
                if activation == 'relu':
                    f.write('ReLU')
                # add more here
                f.write(', ')
                f.write('filters={0}, '.format(layer.filters))
                f.write('stride={0}, '.format(list(layer.strides)))
                f.write('kernel_size={0}, '.format(list(layer.kernel_size)))
                f.write('input_shape={0}, '.format([i for i in layer.input_shape if i is not None]))
                f.write('padding={0}\n'.format(1 if layer.padding == 'same' else 0))
                for w in layer.get_weights():
                    f.write(str(np.ndarray.tolist(w)))
                    f.write('\n')
            elif isinstance(layer, tf.keras.layers.MaxPool2D):
                f.write('MaxPooling2D\n')
                f.write('pool_size={0}, '.format(list(layer.pool_size)))
                f.write('stride={0}, '.format(list(layer.strides)))
                f.write('input_shape={0}\n'.format([i for i in layer.input_shape if i is not None]))
            elif isinstance(layer, tf.keras.layers.Flatten):
                pass # Flattened by Dense layer. See https://github.com/eth-sri/eran/blob/master/tf_verify/read_net_file.py#L117
            elif isinstance(layer, tf.keras.layers.Dense):
                activation = tf.keras.activations.serialize(layer.activation)
                if activation == 'relu':
                    f.write('ReLU')
                elif activation == 'softmax':
                    f.write('Affine') # TODO not strictly true
                f.write('\n')
                for w in layer.get_weights():
                    f.write(str(np.ndarray.tolist(w.transpose())))
                    f.write('\n')

In [0]:
# We are truncated '8'
TRUCATED_PROBs = [0, 0.2, 0.4, 0.6, 0.8, 0.99]
DIFF_LABEL = 8

for TRUCATED_PROB in TRUCATED_PROBs:
    print("TRUCATED_PROB :::",TRUCATED_PROB)

    #Truncating
    indices = [i for i in np.arange(train_labels.shape[0])
    if np.random.random() >= TRUCATED_PROB or train_labels[i] != DIFF_LABEL]

    # Truncate to integer multiple of batch size (due to 
    #https://github.com/tensorflow/privacy/issues/40)
    indices = indices[:(len(indices) // batch_size * batch_size)]

    #indices = indices[np.random.permutation(len(indices))]

    # Filter the train_data and train_labels by those indices
    truncated_train_data = train_data[indices]
    truncated_train_labels = train_labels[indices]
    for i,c in enumerate(np.bincount(truncated_train_labels)):
        print(f'Number of training samples for label {i} :{c}')

    truncated_train_labels_ohe = tf.keras.utils.to_categorical(truncated_train_labels,
                                                        num_classes= 10)
    test_labels_ohe = tf.keras.utils.to_categorical(test_labels, 
                                                num_classes= 10)
    
    #Building the learning model
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    ])

    optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=num_microbatches,
        learning_rate=learning_rate)

    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
    

    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    model.fit(truncated_train_data, truncated_train_labels_ohe,
            epochs=epochs,
            validation_data=(test_data, test_labels_ohe),
            batch_size=batch_size)
    model_path = f'/content/model_ckpts/mnist_TRUCATED_PROB_{TRUCATED_PROB}.tf'
    create_tf_file(model, model_path)


## Building the learning model

In [0]:
for i in range(10):
    i_idx = test_labels == i
    results = model.evaluate(test_data[i_idx], test_labels_ohe[i_idx],
                             batch_size=1, verbose=0)
    print('Accuracy for label ' + str(i) + ': ' + str(results[1]) + '. Number of test samples: ' + str(np.sum(i_idx)))

### Test

In [0]:
%cd /content/ERAN/tf_verify

In [0]:
import os
from google.colab import files

test_out_dir = '/content/test_output/'
if not os.path.exists(test_out_dir):
        os.makedirs(test_out_dir)

def make_data_for_test(digit, test_counts = 50):
    line_counts = 0
    digit_data_dir = f"/content/mnist{digit}/"
    if not os.path.exists(digit_data_dir):
        os.makedirs(digit_data_dir)
    
    #os.mkdir("/content/mnist_splits/")
    with open('/content/sample_data/mnist_test.csv','r') as f:
        with open(f'/content/mnist{digit}/mnist_test_d{digit}.csv','w') as fw:
            for l in f.readlines():
                if l.split(',')[0] == digit:
                    line_counts += 1
                    fw.write(l)
                    if line_counts >= test_counts:
                        break

def data_clean_up(digit):
    digit_data_file = f'/content/mnist{digit}/mnist_test_d{digit}.csv'
    if not os.path.exists(digit_data_file):
        os.remove(digit_data_file)
        os.rmdir(f"/content/mnist{digit}/")

epsilons = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]

def run_robust_tests(digit):
    #os.system("cp /content/sample_data/mnist_test.csv /content/ERAN/data/mnist_test.csv")
    model_files = os.listdir('/content/model_ckpts/')
    model_files = [f for f in model_files if f[-11:] != 'checkpoints']
    for model_f in model_files:
        print(model_f)
        prob = model_f[:-3].split("_")[3]
        for eps in epsilons:
            eps = str('{0:.7f}'.format(eps))
            print(f'=======prob :{prob}  eps :{eps}=======')
            #create output file
            os.system(f'touch /content/test_output/test{digit}_result_prob{prob}_eps{eps}.txt')
            print(f"Data split")
            #copy data to required locations
            os.system(f'cp /content/mnist{digit}/mnist_test_d{digit}.csv /content/ERAN/data/mnist_test.csv')
            os.system(f'cp /content/mnist{digit}/mnist_test_d{digit}.csv /content/ERAN/deepg/code/datasets/mnist_test.csv')

            console_cmd = f'python3 . --netname /content/model_ckpts/{model_f} --epsilon {eps} --domain deepzono \
                            --dataset mnist >> /content/test_output/test{digit}_result_prob{prob}_eps{eps}.txt'
            os.system(console_cmd)

            files.download(f'/content/test_output/test{digit}_result_prob{prob}_eps{eps}.txt')

def run_robust_single_label(digit):
    digit = str(digit)
    make_data_for_test(digit)
    run_robust_tests(digit)
    data_clean_up(digit)

#run_robust_single_label(1)
all_digits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for d in all_digits:
    print(f"####### Processing for: {d}")
    run_robust_single_label(d)


In [0]:
#!python3 . --netname /content/model_ckpts/mnist_TRUCATED_PROB_0.6.tf --epsilon .1 --domain deepzono --dataset mnist > /content/test_output/test_result_prob0.6_eps0.01.txt

## Result Analysis

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

In [0]:
test_out_dir = "/content/test_output/"

def read_results(robust_out):
    '''Reads the output log files from ERAN and returns an array
    1: Verified
    0: Failed to verify
    -1: model classifies wrong'''
    img_counter = 0
    verified_result = []
    with open(robust_out) as f:
        for line in f.readlines():
            if line[:3] != 'img':
                continue

            ws = line.split()
            try:
                if len(ws) >= 3:
                    if ws[2] == 'Verified':
                        verified_result.append(1)
                        img_counter = img_counter + 1
                    elif ws[2] == 'Failed':
                        verified_result.append(0)
                        img_counter = img_counter + 1
                    elif ws[2]+ws[3] == 'notconsidered,':
                        verified_result.append(-1)
                        img_counter = img_counter + 1
                    else:
                        print('Unknown result :::'+line)
                        print(ws[2]+ws[3])
                        raise

            except:
                print("unhandled output:::"+line)
                raise

        return verified_result
    
def get_result(digit):
    TRUCATED_PROBs = [0, 0.2, 0.4, 0.6, 0.8, 0.99]
    epsilons = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
    ver_result_df = pd.DataFrame()
    for prob in TRUCATED_PROBs:
        for eps in epsilons:
            eps = str('{0:.7f}'.format(eps))
            result_file = f'test{digit}_result_prob{prob}_eps{eps}.txt'
            result_file =  test_out_dir + result_file
            ver_result = read_results(result_file)
            col_name = f'prob{prob}_eps{eps}'
            ver_result_df[col_name] = ver_result
            
    return ver_result_df
    
def calculate_accuracy(df, prob, eps):
    eps = str('{0:.7f}'.format(eps))
    n = len(df)
    test_out = df[f'prob{prob}_eps{eps}']
    #results:::  
    #1: correct and robust
    #0: correct and not robust
    #-1: incorrect and not verified
    accurate_count = np.sum(test_out != -1)
    accuracy = 100 * accurate_count/n
    robust_acc = 0.0
    if accurate_count != 0:
        robust_acc = np.sum(test_out == 1)/accurate_count
    
    return accuracy,robust_acc



def get_agg_result(digit, eps = 0.1, measure = 'accuracy'):
    assert measure in ['accuracy','robustness'],"mesure should be either 'accuracy' or 'robustness'"
    j = 0
    if measure == 'robustness':
        j = 1
        
    probs = [0, 0.2, 0.4, 0.6, 0.8, 0.99]
    
    r = [calculate_accuracy(results[digit], prob = p , eps = eps)[j] for p in probs]
    return r

In [0]:
plt.figure(figsize= (8, 6))
probs_x = 100*np.array(probs)
plt.plot(probs_x,get_agg_result(0), label = 0)
plt.plot(probs_x,get_agg_result(1), label = 1)
plt.plot(probs_x,get_agg_result(2), label = 2)
plt.plot(probs_x,get_agg_result(3), label = 3)
plt.plot(probs_x,get_agg_result(4), label = 4)
plt.plot(probs_x,get_agg_result(5), label = 5)
plt.plot(probs_x,get_agg_result(6), label = 6)
plt.plot(probs_x,get_agg_result(7), label = 7)
plt.plot(probs_x,get_agg_result(8), label = 8)
plt.plot(probs_x,get_agg_result(9), label = 9)
plt.legend(loc = 5)
plt.xlabel("Percentage Drop-off Training Label 8")
plt.ylabel("Accuracy")
plt.title("DP Model Accuracy")

In [0]:
def eps_plot(eps, loc = 5):
    plt.figure(figsize= (8, 6))
    probs_x = 100*np.array(probs)
    plt.plot(probs_x,get_agg_result(0, measure = 'robustness', eps = eps), label = 0)
    plt.plot(probs_x,get_agg_result(1, measure = 'robustness', eps = eps), label = 1)
    plt.plot(probs_x,get_agg_result(2, measure = 'robustness', eps = eps), label = 2)
    plt.plot(probs_x,get_agg_result(3, measure = 'robustness', eps = eps), label = 3)
    plt.plot(probs_x,get_agg_result(4, measure = 'robustness', eps = eps), label = 4)
    plt.plot(probs_x,get_agg_result(5, measure = 'robustness', eps = eps), label = 5)
    plt.plot(probs_x,get_agg_result(6, measure = 'robustness', eps = eps), label = 6)
    plt.plot(probs_x,get_agg_result(7, measure = 'robustness', eps = eps), label = 7)
    plt.plot(probs_x,get_agg_result(8, measure = 'robustness', eps = eps), label = 8)
    plt.plot(probs_x,get_agg_result(9, measure = 'robustness', eps = eps), label = 9)
    plt.legend(loc = loc)
    plt.title(f"DP Robustness $\epsilon$ = {eps}")
    plt.xlabel("Percentage Drop-off Training Label 8")
    plt.ylabel("Robustness")
    return(plt)

In [0]:
eps_plot(0.001)